In [1]:
import os
from pathlib import Path

from tfx import v1 as tfx
from tfx.orchestration import data_types
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.orchestration.experimental.interactive.interactive_context import \
    InteractiveContext
from tfx.components import Trainer
from tfx.components.base import executor_spec
# GenericExecutor를 로드하여 학습 실행자를 재정의합니다.
from tfx.components.trainer.executor import GenericExecutor
from tfx.proto import trainer_pb2

import tensorflow as tf
import tensorflow_transform as tft

# 피처 이름, 피처 차원
ONE_HOT_FEATURES = {
    "product": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90,
}

# 피처 이름, 버킷 개수
BUCKET_FEATURES = {"zip_code": 10}

# 피처 이름, 값은 정의되지 않음
TEXT_FEATURES = {"consumer_complaint_narrative": None}

LABEL_KEY = "consumer_disputed"


context = InteractiveContext()

TRAINING_STEPS = 1000
EVALUATION_STEPS = 100

DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'

dir_path = Path().parent.absolute()
data_dir = os.path.join(dir_path, "..", "..", "data", "taxi")

Path(data_dir).mkdir(parents=True, exist_ok=True)

# 웹 상에서 원시 데이터를 다운 받습니다.
filepath = tf.keras.utils.get_file(
    os.path.join(data_dir, "data.csv"),
    DATA_PATH)

In [2]:
example_gen = tfx.components.CsvExampleGen(input_base=data_dir)
context.run(example_gen)

/Users/chris/opt/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:1922812,xor_checksum:1630233420,sum_checksum:1630233420"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [3]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

In [4]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

for artifact in statistics_gen.outputs['statistics'].get():
    print(artifact.uri)


/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/StatisticsGen/statistics/2


In [5]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"],
    infer_feature_shape=False,
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 4
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/SchemaGen/schema/4"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [6]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"],
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/ExampleValidator/anomalies/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [7]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("taxi_transform.py"))
context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Transform/transform_graph/6/.temp_path/tftransform_tmp/e6cadc11737a4c379b1e5c119496d1c8/assets


INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Transform/transform_graph/6/.temp_path/tftransform_tmp/e6cadc11737a4c379b1e5c119496d1c8/assets


INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Transform/transform_graph/6/.temp_path/tftransform_tmp/369a63a9ad794761956127341c67972d/assets


INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Transform/transform_graph/6/.temp_path/tftransform_tmp/369a63a9ad794761956127341c67972d/assets


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 22
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Transform/transform_graph/6"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Transform/transformed_examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 23
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Transform/updated_analyzer_cache/6"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Transform/pre_transform_schema/6"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
         

In [8]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath("taxi_trainer.py"),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer)

/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


 5706/10000 [================>.............] - ETA: 2:22:24 - loss: 0.6719 - binary_accuracy: 0.80 - ETA: 4:22 - loss: 0.6621 - binary_accuracy: 0.7417   - ETA: 33s - loss: 0.6238 - binary_accuracy: 0.772 - ETA: 26s - loss: 0.5776 - binary_accuracy: 0.77 - ETA: 23s - loss: 0.5477 - binary_accuracy: 0.78 - ETA: 21s - loss: 0.5353 - binary_accuracy: 0.77 - ETA: 19s - loss: 0.5277 - binary_accuracy: 0.77 - ETA: 18s - loss: 0.5208 - binary_accuracy: 0.77 - ETA: 17s - loss: 0.5151 - binary_accuracy: 0.77 - ETA: 17s - loss: 0.5067 - binary_accuracy: 0.77 - ETA: 16s - loss: 0.5021 - binary_accuracy: 0.77 - ETA: 16s - loss: 0.4960 - binary_accuracy: 0.77 - ETA: 16s - loss: 0.4900 - binary_accuracy: 0.77 - ETA: 15s - loss: 0.4864 - binary_accuracy: 0.77 - ETA: 15s - loss: 0.4818 - binary_accuracy: 0.77 - ETA: 15s - loss: 0.4771 - binary_accuracy: 0.77 - ETA: 15s - loss: 0.4725 - binary_accuracy: 0.77 - ETA: 15s - loss: 0.4680 - binary_accuracy: 0.77 - ETA: 15s - loss: 0.4628 - binary_accuracy: 

10000/10000 [==============================] - ETA: 6s - loss: 0.2660 - binary_accuracy: 0.838 - ETA: 6s - loss: 0.2656 - binary_accuracy: 0.838 - ETA: 6s - loss: 0.2652 - binary_accuracy: 0.839 - ETA: 5s - loss: 0.2648 - binary_accuracy: 0.839 - ETA: 5s - loss: 0.2645 - binary_accuracy: 0.839 - ETA: 5s - loss: 0.2642 - binary_accuracy: 0.840 - ETA: 5s - loss: 0.2637 - binary_accuracy: 0.840 - ETA: 5s - loss: 0.2633 - binary_accuracy: 0.840 - ETA: 5s - loss: 0.2630 - binary_accuracy: 0.841 - ETA: 5s - loss: 0.2626 - binary_accuracy: 0.841 - ETA: 5s - loss: 0.2622 - binary_accuracy: 0.841 - ETA: 5s - loss: 0.2618 - binary_accuracy: 0.841 - ETA: 5s - loss: 0.2615 - binary_accuracy: 0.842 - ETA: 5s - loss: 0.2611 - binary_accuracy: 0.842 - ETA: 5s - loss: 0.2608 - binary_accuracy: 0.842 - ETA: 5s - loss: 0.2605 - binary_accuracy: 0.843 - ETA: 5s - loss: 0.2601 - binary_accuracy: 0.843 - ETA: 5s - loss: 0.2598 - binary_accuracy: 0.843 - ETA: 5s - loss: 0.2595 - binary_accuracy: 0.843 - ETA

INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 13
        type_id: 25
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Trainer/model/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 14
        type_id: 26
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T02_46_38.431738-_eg5eeku/Trainer/model_run/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))